# Calculation of E3FP fingerprints

## Requirements:
1. Ligprep SDF files containing SMILES column genetated in Schrodinger maestro.

In [ ]:
import os
import time
import fileinput
import pandas as pd
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed
from rdkit.Chem import PandasTools as pt
from e3fp.pipeline import fprints_from_smiles

In [ ]:
def e3fp(path):
    '''Generating a dataframe containing e3fp fingerpints.'''
    start_time = time.time()
    
    def loadSDF(path):
        global sdf
        sdf = pt.LoadSDF(path)
        sdf = sdf.drop(["ROMol"],axis=1)
        return sdf
    loadSDF(path)
    
    def Calculation(self):
    # Setting the configuration for generating fingerprint.
        fprint_params = {'bits': 4096, 'radius_multiplier': 1.5, 'rdkit_invariants': True}
        confgen_params = {'max_energy_diff': 1.0, 'first': 0}
        global fingerprints
        fingerprints = fprints_from_smiles(self, "test", confgen_params=confgen_params, fprint_params=fprint_params)
        return fingerprints
    
    def Parallelizer(sdf):
        global job
        job = Parallel(n_jobs=(multiprocessing.cpu_count()-1),timeout=None)(delayed(Calculation)(smile) for smile in tqdm(sdf['s_user_SMILES']))
        return job
    e3fp_fingerprints = Parallelizer(sdf)
    
    def ProcessFingerprints(e3fp_fingerprints):
        print(e3fp_fingerprints)
        fin = pd.DataFrame()
        fin['e3fp'] = e3fp_fingerprints
        fin.to_csv(f"{os.path.dirname(path)}/intermediate.csv",index=False)
        
        with fileinput.FileInput(f"{os.path.dirname(path)}/intermediate.csv", inplace=True, backup=".bak") as file:
            for line in file:
                print(line.replace("[Fingerprint(indices=array([",""),end='')

        with fileinput.FileInput(f"{os.path.dirname(path)}/intermediate.csv", inplace=True, backup=".bak") as file:
            for line in file:
                print(line.replace("]), level=5, bits=4096, name=test_0)]",""),end='')
        
        global df
        df = pd.read_csv(f"{os.path.dirname(path)}/intermediate.csv")
        df2 = df["e3fp"].str.split(',', expand=True)
        df = df.drop(['e3fp'],axis=1)
        df = sdf.join(df2)
        df.fillna(0, inplace=True)
        df.to_csv(f"{os.path.dirname(path)}/{os.path.splitext(os.path.basename(path))[0]}-e3fp.csv",index=False)
        return df
    ProcessFingerprints(e3fp_fingerprints)
    print(f"Time taken to generate e3fp fingerprints for {len(df)} molecules is {(time.time()-start_time)/60} minutes.")

In [ ]:
e3fp("/home2/satvik/ml_dl/oral-bioavailability/test_1.sdf")